In [1]:
import gc
import numpy
import os
import pandas
import tensorflow

from datetime import *
from sklearn.model_selection import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.regularizers import *

In [2]:
# Set NVIDIA GeForce RTX 3060 memory limit in MB
GPU_LIM_MB = 1_024 * 20

In [3]:
# Retrieve a list of available GPUs
gpus = tensorflow.config.list_physical_devices('GPU')
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
# Set the memory limit for the first GPU
if gpus:
    try:
        tensorflow.config.set_visible_devices(gpus[0], 'GPU')
        tensorflow.config.experimental.set_virtual_device_configuration(gpus[0], [tensorflow.config.experimental.VirtualDeviceConfiguration(memory_limit=GPU_LIM_MB)])
    except RuntimeError as e:
        print(e)

In [5]:
# Load data
df = pandas.read_csv('dark.csv')
df.head()

,id,board,side,piece,atk,move,river,trap,den,score,winner
0,3c303edc-80ac-422b-bf2c-8668724fdbf2,l-r---E-T-d-----C---p---W-------------w---P---...,-1,R,1,G7F7,0,0,0,0,0
1,3c303edc-80ac-422b-bf2c-8668724fdbf2,l--r--E-T-d-----C---p---W-------------w---P---...,1,r,1,A3A4,0,0,0,0,0
2,3c303edc-80ac-422b-bf2c-8668724fdbf2,l--r--E-T-d-----C---p---W-------------w---P---...,-1,R,1,F7G7,0,0,0,0,0
3,3c303edc-80ac-422b-bf2c-8668724fdbf2,l---r-E-T-d-----C---p---W-------------w---P---...,1,r,1,A4A5,0,0,0,0,0
4,3c303edc-80ac-422b-bf2c-8668724fdbf2,l---r--ET-d-----C---p---W-------------w---P---...,-1,E,8,A7A8,0,0,0,0,0


In [6]:
# Calculate the total number of rows
count = len(df)
count

33093425

In [7]:
# Extract the sample at index 32
sample = df.iloc[32]
sample

id                     3c303edc-80ac-422b-bf2c-8668724fdbf2
board     l----W--E-d---r-C---p-----T------P----w----tLc...
side                                                     -1
piece                                                     L
atk                                                       7
move                                                   F9E9
river                                                     0
trap                                                     -1
den                                                       0
score                                                    30
winner                                                    0
Name: 32, dtype: object

In [8]:
# Encode chess pieces to integer
def encode_piece(piece_char):
    piece_mapping = {'-': 0, 'r': 1, 'c': 2, 'd': 3, 'w': 4, 'p': 5, 't': 6, 'l': 7, 'e': 8, 'R': -1, 'C': -2, 'D': -3, 'W': -4, 'P': -5, 'T': -6, 'L': -7, 'E': -8}
    return piece_mapping.get(piece_char, 0)

encode_piece(sample['piece'])

-7

In [9]:
# Encode chess board to matrix
def encode_board(board_str):
    board_matrix = numpy.zeros((9, 7))
    for i, piece in enumerate(board_str[::-1]):
        row, col = divmod(i, 9)
        board_matrix[col][row] = encode_piece(piece)
    return numpy.flip(numpy.flip(board_matrix, 0), 1)

encode_board(sample['board'])

array([[ 7.,  0.,  0.,  0.,  0.,  2.,  0.],
       [ 0.,  3.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  5.,  0.,  4.,  8.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0., -1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-4.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0., -5.,  0.,  0.,  0.],
       [ 0., -2.,  0.,  0.,  6.,  0.,  0.],
       [-8.,  0., -6.,  0., -7.,  0.,  0.]])

In [10]:
# Encode all chess boards
df['board_encoded'] = df['board'].apply(encode_board)
board_matrix_flattened = numpy.array(df['board_encoded'].tolist()).reshape(count, -1)
board_matrix_flattened

array([[ 7.,  0.,  0., ...,  0.,  0., -7.],
       [ 7.,  0.,  0., ...,  0.,  0., -7.],
       [ 7.,  0.,  0., ...,  0.,  0., -7.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [11]:
# Prepare data for model training
X = numpy.array(df['board_encoded'].tolist()).reshape(count, -1)
y = df['score'].values
X.shape, y.shape

((33093425, 63), (33093425,))

In [12]:
# Reset Keras session
def reset_keras():
    tensorflow.keras.backend.clear_session()
    gc.collect()

In [13]:
# Build the model architecture
def build_model(input_shape, activation='relu'):
    # Create a sequential model
    model = Sequential([
        # Input layer specifies the shape of the input data
        Input(shape=input_shape),
        # First convolution layer with 128 filters
        Conv2D(128, (3, 3), padding='same'),
        BatchNormalization(),
        Activation(activation),
        MaxPooling2D((2, 2)),
        # Second convolution layer with 256 filters
        Conv2D(256, (3, 3), padding='same'),
        BatchNormalization(),
        Activation(activation),
        MaxPooling2D((2, 2)),
        # Flatten the output from 2D to 1D before passing to the dense layer
        Flatten(),
        Dense(2048, activation='relu', kernel_regularizer=l2(0.01)),
        Dropout(0.5),
        # Output layer with linear activation to predict a continuous value
        Dense(1, activation='linear')
    ])
    # Compile the model with Adam optimizer and mean squared error loss
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error', metrics=['mae'])
    # Return the model
    return model

In [14]:
# Create the neural network model with the specified input shape and activation function
model = build_model((9, 7, 1), activation='leaky_relu')

In [15]:
# Set up TensorBoard logging with a timestamped directory to monitor the training process
log_dir = os.path.join("logs", "fit", datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [16]:
# Initialize callbacks for adaptive learning rate, early stopping to prevent overfitting, and saving the best model
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint("best_model.h5", monitor='val_loss', save_best_only=True, save_format='tf')

In [17]:
X.shape, y.shape

((33093425, 63), (33093425,))

In [18]:
# Initialize an empty list for storing results and a KFold object for 5-fold cross-validation
results = []
kf = KFold(n_splits=5)

# Cross-validation to evaluate model
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    reset_keras()
    history = model.fit(X_train.reshape(-1, 9, 7, 1), y_train, epochs=50, batch_size=256, validation_data=(X_test.reshape(-1, 9, 7, 1), y_test), callbacks=[reduce_lr, early_stopping, checkpoint, tensorboard_callback])
    reset_keras()
    results.append(model.evaluate(X_test.reshape(-1, 9, 7, 1), y_test))

# Output the results of cross-validation
print("Fold results:", results)
print("Average result:", numpy.mean(results, axis=0))

Epoch 1/50
103417/103417 [==============================] - 569s 5ms/step - loss: 222.3724 - mae: 7.1522 - val_loss: 83.6308 - val_mae: 3.9706 - lr: 1.0000e-04
Epoch 2/50
103417/103417 [==============================] - 532s 5ms/step - loss: 92.3235 - mae: 5.0864 - val_loss: 57.6835 - val_mae: 3.6376 - lr: 1.0000e-04
Epoch 3/50
103417/103417 [==============================] - 532s 5ms/step - loss: 72.5608 - mae: 4.5535 - val_loss: 64.3201 - val_mae: 4.7631 - lr: 1.0000e-04
Epoch 4/50
103417/103417 [==============================] - 515s 5ms/step - loss: 61.6155 - mae: 4.2123 - val_loss: 34.5451 - val_mae: 2.6118 - lr: 1.0000e-04
Epoch 5/50
103417/103417 [==============================] - 510s 5ms/step - loss: 54.1167 - mae: 3.9386 - val_loss: 37.0597 - val_mae: 2.9990 - lr: 1.0000e-04
Epoch 6/50
103417/103417 [==============================] - 511s 5ms/step - loss: 48.5454 - mae: 3.7196 - val_loss: 30.1525 - val_mae: 2.4568 - lr: 1.0000e-04
Epoch 7/50
103417/103417 [===================